In [7]:
## 
## verzió B  csak létezés ellenőrzés, de a lehetséges alternatívákkal összevetve
##

import pandas as pd
import numpy as np

from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from plot_keras_history import plot_history
import random 
import math

def define_discriminator(att_genom, n_outputs):
    n_inputs = sum(att_genom)
    model = Sequential()
    model.add(Dense(8 *n_inputs, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(8 *n_inputs, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(n_outputs, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def train_discrimantor(discriminator,att_genom, DA, DD, M, N2, E, Ci):
    n_inputs = sum(att_genom)
    data_dim = len(att_genom)
    
    N = DA.shape[0]
    x_train = np.zeros((N*N2,n_inputs))
    y_train = np.zeros((N*N2,M))
    
    n = 0
    for _ in range(N2):
        indx = random.sample(range(N),N)
        for i in indx:
            m = indx[i]
            k = 0
            for j in range(data_dim):
                if att_genom[j] == 1:
                    x_train[n,k] = DA[m,j]
                    k = k + 1
                if j == Ci:
                    y_train[n, int(DA[m,j]) ] = 1
            n = n + 1

    r = discriminator.fit(x_train, y_train,epochs=E,verbose = 0)
    # plot_history(r.history)

def prepare_data(M, Nt):

    rdata0 = pd.read_csv("BankChurners.csv")
    rdata = rdata0.drop([rdata0.columns[0],rdata0.columns[-1],rdata0.columns[-2]],axis=1)
    rdata.head()
    if Nt > 1000:
        Nt = 1000
        
    fbe = open("INDX.TXT","r")
    data = fbe.read().split("\n")
    inx2 = [int(x) for x in data if len(x) > 0]
    fbe.close()    
    indx = inx2[:Nt]
    #indx = random.sample(range(rdata.shape[0]), Nt)
    
    rindx = []
    for i in indx:
        rindx.append(rdata.index[i])
        
    print (rdata.shape)
    rdata = rdata.drop(rindx)
    print (rdata.shape)
    
    
    Dicts = []
    for c in range(len(rdata.columns)):
        l1 = rdata[rdata.columns[c]]
        l2 = list(dict.fromkeys(l1))
        l2.sort()
        nn = len(l2)
        Dicts.append(dict())
        for i in range(len(l2)):
            Dicts[c] [l2[i]] = i / nn 
    
    rM = 1./M

    DArray = np.zeros(  (rdata.shape[0], rdata.shape[1]),dtype = "float")
    DD = dict()
    
    for n in range(rdata.shape[0]):
        for c in range(rdata.shape[1]):
            #print (rdata [n,c])
            DArray[n,c] =  ( Dicts[c] [ rdata.iat [n,c]]) // rM
        DD[tuple(DArray[n,:]) ] = 1

        
    print ('A', DArray.shape,  len(DD))
           
    ###########################################
    
    
    rdata0 = pd.read_csv("BankChurners.csv")
    rdata = rdata0.drop([rdata0.columns[0],rdata0.columns[-1],rdata0.columns[-2]],axis=1)
    rdata.head()
    
    indx = random.sample(range(rdata.shape[0]), Nt)
    rindx = []
    for i in indx:
        rindx.append(i)
        
    print (rdata.shape)
    rdata = rdata.iloc[rindx]
    print (rdata.shape)
    
    
    Dicts = []
    for c in range(len(rdata.columns)):
        l1 = rdata[rdata.columns[c]]
        l2 = list(dict.fromkeys(l1))
        l2.sort()
        nn = len(l2)
        Dicts.append(dict())
        for i in range(len(l2)):
            Dicts[c] [l2[i]] = i / nn 
    
    rM = 1./M

    TArray = np.zeros(  (rdata.shape[0], rdata.shape[1]),dtype = "float")
    TD = dict()
    
    for n in range(rdata.shape[0]):
        for c in range(rdata.shape[1]):
            #print (rdata [n,c])
            TArray[n,c] =  ( Dicts[c] [ rdata.iat [n,c]]) // rM
        TD[tuple(TArray[n,:]) ] = 1
        
    print ('B', DArray.shape,  len(DD), TArray.shape,  len(TD))
    return (DArray,DD,TArray,TD)


def test_discriminator(discriminator, att_genom, DA, DD,   M , Ci):
    n_inputs = sum(att_genom)
    data_dim = len(att_genom)
    
    N = DA.shape[0]
    #print ("test",N)
    x_test = np.zeros((N,n_inputs))
    y_test = []
        
    for n in range(N):
        k = 0
        for j in range(data_dim):
            if att_genom[j] == 1:
                x_test[n,k] = DA[n,j]
                k = k + 1
            if j == Ci:
                y_test.append(int(DA[n,j]))
        n = n + 1

    y_pred = discriminator.predict(x_test)
    
    db  = 0
    for n in range(N):
        yy =list(y_pred[n,:])
        pw = yy.index(max(yy))
        if pw == y_test[n]:
            db = db + 1
            
               
    #print ("accuracy=", db / N )
    
    return (db / N)
    
            


In [1]:
M = 10    # values
Ci = 3    # investigated attribute
N3 = 1000   # test size

(DArray, DDict, TArray, TDict)  = prepare_data(M, N3)
N = DArray.shape[0]
data_dim = DArray.shape[1]

N2 = 10  # number of training loops
E = 12  # Epochs
B = 20  # batch 

print ("Discriminator training")
print ("M:",M,"dim:",data_dim, "cube size:", M**data_dim, "real size:",N, " train_size:", N2,"epochs:", E, "test_size:", N3, "mode: real data")

att_genom = [1 for _ in range(data_dim)]
att_genom[Ci] = 0
print (att_genom)
discriminator = define_discriminator(att_genom, M)

train_discrimantor(discriminator, att_genom,  DArray, DDict, M, N2, E, Ci)

test_discriminator(discriminator,  att_genom,  TArray, TDict, M, Ci)



for j in range(data_dim):
    #atts = random.sample(range(data_dim), int(.9*data_dim) )
    #att_genom = [0 for _ in range(data_dim)]
    #for i in atts:
    #    att_genom[i] = 1    
    #att_genom[Ci] = 0
    if j == Ci:
        continue
    att_genom = [0 for _ in range(data_dim)]
    att_genom[j] = 1
    att_genom[Ci] = 0
    print (att_genom)
    discriminator = define_discriminator(att_genom, M)

    train_discrimantor(discriminator, att_genom,  DArray, DDict, M, N2, E, Ci)
    
    test_discriminator(discriminator,  att_genom,  TArray, TDict, M, Ci)
    

NameError: name 'prepare_data' is not defined

In [5]:
## GA2

M = 10    # values
Ci = 3    # investigated attribute
N3 = 1000   # test size

(DArray, DDict, TArray, TDict)  = prepare_data(M, N3)
N = DArray.shape[0]
data_dim = DArray.shape[1]

N2 = 10  # number of training loops
E = 12  # Epochs
B = 20  # batch 

genom_pool = dict()

max_acc = 0
sum_acc = 0
best_genom = None
for j in range(data_dim):
    att_genom = [1 for _ in range(data_dim)]
    att_genom[j] = 0
    att_genom[Ci] = 0

    discriminator = define_discriminator(att_genom, M)

    train_discrimantor(discriminator, att_genom,  DArray, DDict, M, N2, E, Ci)
    
    acc = test_discriminator(discriminator,  att_genom,  TArray, TDict, M, Ci)    

    genom_pool[tuple(att_genom)] =  [acc,3*6]
    sum_acc += acc
    print ("acc=",acc,max_acc)
    if acc > max_acc:
        max_acc = acc
        best_genom = att_genom.copy()
        print (best_genom)

while (len(genom_pool) > 1) :
    print (max_acc, len(genom_pool) , sum_acc / len(genom_pool), best_genom)
    #############################
    #mtc = random.sample(range(len(genom_pool) ),2)
    #key1 = list(genom_pool.keys())[mtc[0]]
    #key2 = list(genom_pool.keys())[mtc[1]]
    L = [(k,genom_pool[k]) for k in genom_pool.keys()]
    B = [x[1][0] for x in L]
    s = sum (B)
    for i in range(1,len(B)):
        B[i] = B[i] + B[i-1]
    for i in range(len(B)):
        B[i] = B[i]/s
    x1 = random.random()
    p = 0
    while (B[p] < x):
        p = p + 1
    key1 = L[p][0]
    x2 = random.random()
    p = 0
    while (B[p] < x):
        p = p + 1
    key2 = L[p][0]
    #########################
    
    att_genom = list(key1).copy()
    for i in range(data_dim):
        if list(key2)[i] == 0:
            att_genom[i] = 0
    
    fnd = 0
    if tuple(att_genom) in genom_pool.keys():
        acc = genom_pool[tuple(att_genom)][0]
        fnd = 1
    else:
        discriminator = define_discriminator(att_genom, M)

        train_discrimantor(discriminator, att_genom,  DArray, DDict, M, N2, E, Ci)

        acc = test_discriminator(discriminator,  att_genom,  TArray, TDict, M, Ci)    

    if acc > max_acc:
        max_acc = acc
        best_genom = att_genom.copy()
        genom_pool[key1][1] -= 1
        genom_pool[key2][1] -= 1
        if genom_pool[key1][1] <= 0:
            sum_acc -= genom_pool[key1][0]
            del genom_pool[key1]
        if genom_pool[key2][1] <= 0:
            sum_acc -= genom_pool[key2][0]
            del genom_pool[key2]
        genom_pool[tuple(att_genom)] = [acc,3*6]
        if fnd == 0:
            sum_acc += acc
    else:
        if acc > sum_acc / len(genom_pool) :
            genom_pool[key1][1] -= 2
            genom_pool[key2][1] -= 2
            if genom_pool[key1][1] <= 0:
                sum_acc -= genom_pool[key1][0]
                del genom_pool[key1]
            if genom_pool[key2][1] <= 0:
                sum_acc -= genom_pool[key2][0]
                del genom_pool[key2]
            if random.random () > .5 :
                genom_pool[tuple(att_genom)] = [acc,3*6]
                if fnd == 0:
                    sum_acc += acc
    
        else:
            genom_pool[key1][1] -= 3
            genom_pool[key2][1] -= 3
            if genom_pool[key1][1] <= 0:
                sum_acc -= genom_pool[key1][0]
                del genom_pool[key1]
            if genom_pool[key2][1] <= 0:
                sum_acc -= genom_pool[key2][0]
                del genom_pool[key2]
            

(10127, 20)
(9127, 20)
A (9127, 20) 9126
(10127, 20)
(1000, 20)
B (9127, 20) 9126 (1000, 20) 1000
acc= 0.565 0
[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
acc= 0.577 0.565
[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
acc= 0.579 0.577
[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
acc= 0.594 0.579
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
acc= 0.532 0.594
acc= 0.573 0.594
acc= 0.545 0.594
acc= 0.578 0.594
acc= 0.572 0.594
acc= 0.555 0.594
acc= 0.557 0.594
acc= 0.561 0.594
acc= 0.601 0.594
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
acc= 0.545 0.601
acc= 0.616 0.601
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
acc= 0.593 0.616
acc= 0.566 0.616
acc= 0.557 0.616
acc= 0.55 0.616
acc= 0.625 0.616
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
0.625 20 0.5720500000000002 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
0.625 20 0.5720500000000002 [1, 1, 1, 0, 1, 1, 1, 1

KeyboardInterrupt: 

In [3]:
## GA
M = 10    # values
Ci = 3    # investigated attribute
N3 = 1000   # test size

(DArray, DDict, TArray, TDict)  = prepare_data(M, N3)
N = DArray.shape[0]
data_dim = DArray.shape[1]

N2 = 10  # number of training loops
E = 12  # Epochs
B = 20  # batch 

genom_pool = dict()

max_acc = 0
sum_acc = 0
best_genom = None
for j in range(data_dim):
    if j == Ci:
        continue
    att_genom = [0 for _ in range(data_dim)]
    att_genom[j] = 1
    att_genom[Ci] = 0

    discriminator = define_discriminator(att_genom, M)

    train_discrimantor(discriminator, att_genom,  DArray, DDict, M, N2, E, Ci)
    
    acc = test_discriminator(discriminator,  att_genom,  TArray, TDict, M, Ci)    

    genom_pool[tuple(att_genom)] =  [acc,3*6]
    sum_acc += acc
    print ("acc=",acc,max_acc)
    if acc > max_acc:
        max_acc = acc
        best_genom = att_genom.copy()
        print (best_genom)

while (len(genom_pool) > 1) :
    print (max_acc, len(genom_pool) , sum_acc / len(genom_pool), best_genom)
    mtc = random.sample(range(len(genom_pool) ),2)
    key1 = list(genom_pool.keys())[mtc[0]]
    key2 = list(genom_pool.keys())[mtc[1]]
    att_genom = list(key1).copy()
    for i in range(data_dim):
        if list(key2)[i] == 1:
            att_genom[i] = 1
    
    discriminator = define_discriminator(att_genom, M)

    train_discrimantor(discriminator, att_genom,  DArray, DDict, M, N2, E, Ci)
    
    acc = test_discriminator(discriminator,  att_genom,  TArray, TDict, M, Ci)    

    if acc > max_acc:
        max_acc = acc
        best_genom = att_genom.copy()
        genom_pool[key1][1] -= 1
        genom_pool[key2][1] -= 1
        if genom_pool[key1][1] <= 0:
            sum_acc -= genom_pool[key1][0]
            del genom_pool[key1]
        if genom_pool[key2][1] <= 0:
            sum_acc -= genom_pool[key2][0]
            del genom_pool[key2]
        genom_pool[tuple(att_genom)] = [acc,3*6]
        sum_acc += acc
    else:
        if acc > sum_acc / len(genom_pool) :
            genom_pool[key1][1] -= 2
            genom_pool[key2][1] -= 2
            if genom_pool[key1][1] <= 0:
                sum_acc -= genom_pool[key1][0]
                del genom_pool[key1]
            if genom_pool[key2][1] <= 0:
                sum_acc -= genom_pool[key2][0]
                del genom_pool[key2]
            if random.random () > .5 :
                genom_pool[tuple(att_genom)] = [acc,3*6]
                sum_acc += acc
    
        else:
            genom_pool[key1][1] -= 3
            genom_pool[key2][1] -= 3
            if genom_pool[key1][1] <= 0:
                sum_acc -= genom_pool[key1][0]
                del genom_pool[key1]
            if genom_pool[key2][1] <= 0:
                sum_acc -= genom_pool[key2][0]
                del genom_pool[key2]
            

(10127, 20)
(9127, 20)
A (9127, 20) 9126
(10127, 20)
(1000, 20)
B (9127, 20) 9126 (1000, 20) 1000
acc= 0.282 0
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
acc= 0.335 0.282
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
acc= 0.239 0.335
acc= 0.282 0.335
acc= 0.281 0.335
acc= 0.282 0.335
acc= 0.282 0.335
acc= 0.32 0.335
acc= 0.266 0.335
acc= 0.261 0.335
acc= 0.282 0.335
acc= 0.287 0.335
acc= 0.273 0.335
acc= 0.281 0.335
acc= 0.282 0.335
acc= 0.248 0.335
acc= 0.239 0.335
acc= 0.282 0.335
acc= 0.282 0.335
0.335 19 0.2782105263157895 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.341 20 0.28135000000000004 [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.341 20 0.28135000000000004 [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.341 20 0.28135000000000004 [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.341 21 0.28176190476190477 [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.341 21 0.28

0.379 45 0.35188888888888886 [0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
0.379 44 0.3534772727272727 [0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
0.379 44 0.3534772727272727 [0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
0.382 45 0.3541111111111111 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
0.382 45 0.3541111111111111 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
0.382 45 0.3541111111111111 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
0.382 45 0.3541111111111111 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
0.382 44 0.3562272727272727 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
0.382 44 0.3562272727272727 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
0.382 44 0.3562272727272727 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
0.382 44 0.3562272727272727 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
0.382 43 0.358953488

0.472 20 0.5151000000000001 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]
0.472 20 0.5151000000000001 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]
0.472 20 0.5151000000000001 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]
0.472 19 0.5237368421052632 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]
0.472 19 0.5237368421052632 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]
0.472 19 0.5237368421052632 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]
0.472 19 0.5237368421052632 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]
0.472 19 0.5237368421052632 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]
0.472 17 0.5454705882352942 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]
0.472 17 0.5454705882352942 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]
0.472 16 0.5572500000000001 [0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]
0.472 15 0.5689333333

In [9]:

N2 = 10  # number of training loops
E = 20  # Epochs
B = 20  # batch 

print ("Discriminator training")

att_genom = [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   # 0.635
att_genom = [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
#att_genom = [1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
#att_genom = [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
att_genom[Ci] = 0
discriminator = define_discriminator(att_genom, M)

train_discrimantor(discriminator, att_genom,  DArray, DDict, M, N2, E, Ci)

test_discriminator(discriminator,  att_genom,  TArray, TDict, M, Ci)


Discriminator training


0.474

In [ ]:
'''
FULL test
sok véletlen elem a mintaválasztás miatt
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
: 0.591

[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
0.386 --> 0.575,583
[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0.432 --> 0.579.0.587,0.55, 0.57
[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0.388 --> 0.576,0.585

'''

'''
MINI test

[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.401, 0.405
[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.388,0.381
[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.422,0.381
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.41,0.409
[1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.401,0.384
[1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.406,0.455
[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.432,0.4
[1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.431, 0.396
[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.418,0.405
[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.401, 0.412
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.425,0.399
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.414
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.386
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.42
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
test 1000
accuracy= 0.409
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
test 1000
accuracy= 0.417
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
test 1000
accuracy= 0.396
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
test 1000
accuracy= 0.417
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
test 1000
accuracy= 0.379
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
test 1000
accuracy= 0.438
1
​
2
N2 = 10  # number of training loops
'''

In [24]:
mh = MinHeap()
mh.insert([-.6,[1,1,0,1]])
mh.insert([-.2,[1,0,0,1]])
x = mh.delete_min()
print (x)
x = mh.delete_min()
print (x)


[-0.6, [1, 1, 0, 1]]
[-0.2, [1, 0, 0, 1]]
